In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from common import *
import qgrid

### Balance

In [ ]:
b = Balance()
print("Currencies",  b.currencies)
print("Entries", b)
b.add_currency(c.ETH)
b.update(c.ETH, delta_free=1.0, delta_used=0.0)
b.get(c.ETH)[BalanceType.FREE]
Balance.from_dict(b.to_dict())

### Exchange

In [ ]:
# Public informaiton
exchanges = [c.PAPER, c.BINANCE, c.GDAX]#, c.POLONIEX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    print(exchange.timeframes)
    exchange.get_markets()
    exchange.fetch_ohlcv(a, Timeframe.ONE_MIN.value['id'])
    exchange.fetch_order_book(a)
    exchange.fetch_public_trades(a)
    exchange.fetch_ticker(a)

In [ ]:
# Account Information
exchanges = [c.PAPER, c.BINANCE, c.GDAX]
a = Asset(c.ETH, c.BTC)
for ex in exchanges:
    print("Exchange", ex)
    exchange = load_exchange(ex)
    b = exchange.fetch_balance()
    print(exchange.fetch_balance())
    print(exchange.fetch_orders(a))
    print(exchange.fetch_open_orders(a))
    print(exchange.fetch_closed_orders(a))

In [ ]:
exchange = load_exchange(c.PAPER) # c.BINANCE
a = Asset(c.ETH, c.BTC)

# Market BUY
print("Exchange", exchange.id)
order_dict = exchange.create_market_buy_order(a, .01)
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
print(order_dict)

In [ ]:
# Check Order
balance = exchange.fetch_balance()
print(a.base, balance.get(a.base), a.quote, balance.get(a.quote))
found_order = exchange.fetch_order(order_dict['id'], a.symbol)
print("Placed order", found_order) 

In [ ]:
# Sell all remaining quantity
curr_balance = exchange.fetch_balance().get(a.base)
print(a.base, curr_balance[BalanceType.TOTAL])
order = exchange.create_market_sell_order(a, curr_balance[BalanceType.TOTAL])
print(order)

In [ ]:
print(exchange.fetch_balance().get(a.base), exchange.fetch_balance().get(a.quote))

### OHLCV Data

In [ ]:
base = c.ETH
quote = c.BTC
exchange = load_exchange(c.BINANCE)
asset = Asset(base, quote)
assets = [ Asset(coin, quote) for coin in [c.ETH, c.LTC] ]
period = Timeframe.ONE_MIN
start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)

In [ ]:
# Single Coin
df = ohlcv.fetch_and_save_ohlcv_data(exchange, asset, period.value['id'], start, end)
df.head()

In [ ]:
# Load from File
fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period.value['id'])
df = ohlcv.load_chart_data_from_file(fpath)

In [ ]:
# Multiple Coins
ohlcv.download_chart_data(exchange, assets, period.value['id'], start, end)
df = ohlcv.load_multiple_assets(exchange.id, assets, period.value['id'], start)
df.head()

### DataStore

In [ ]:
experiment_name = 'mystrategy1'
fname = 'multiasset'
store = FileStore(os.path.join(cfg.DATA_DIR, experiment_name))

In [ ]:
# DataFrame --> CSV
store.df_to_csv(df, fname)
df = store.csv_to_df(fname, index='time_epoch')
df.head()

In [ ]:
# DataFrame --> JSON
store.df_to_json(df, fname)
df = store.json_to_df(fname, index='time_epoch')
df.head()

In [ ]:
# JSON
dct = {
    'sample': 1,
    'time': datetime.datetime.utcnow()
}
store.save_json(fname, dct)
dct = store.load_json(fname)
dct

### Data Feed

In [ ]:
def get_test_live_feed(exchange_id):
    exchange = load_exchange(exchange_id)
    asset = Asset(c.ETH, c.BTC)
    period = Timeframe.ONE_MIN
    start = datetime.datetime.utcnow() - datetime.timedelta(hours=2)
    end = datetime.datetime.utcnow() - datetime.timedelta(hours=0)
    feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period.value['id'])
    feed = ExchangeDataFeed(exchange, [asset], period, feed_fpath, start, end)
    feed.initialize()
    return feed

def get_test_csv_feed(exchange_id):
    exchange = load_exchange(exchange_id)
    start = datetime.datetime(year=2017, month=1, day=1)
    end = datetime.datetime(year=2018, month=1, day=1)
    asset = Asset(c.ETH, c.BTC)
    period = Timeframe.THIRTY_MIN
    feed_fpath = ohlcv.get_price_data_fpath(asset, exchange.id, period.value['id'])
    ohlcv.fetch_and_save_ohlcv_data(exchange, asset, period.value['id'], start, end)
    feed = CSVDataFeed(feed_fpath)
    feed.initialize()
    return feed

In [ ]:
# CSV Feed
csv_feed = get_test_csv_feed(c.PAPER)

# Grab 1 row at a time
for i in range(3):
    data = feed.next()
    print(data['time_utc'], data['close'])    
    
# Access all rows in history
feed.history().head()

In [ ]:
# Live Feed
live_feed = get_test_live_feed(c.BINANCE)

# Grab 1 row at a time (don't refresh data)
for i in range(3):
    data = feed.next(refresh=False)
    print(data['time_utc'], data['close'])

# Refresh data before next query
data = feed.next(refresh=True)
print(data['time_utc'], data['close'])
    
# Access History
feed.history(t_minus=3).tail()

### OrderType

In [ ]:
assert OrderType.from_type_side('limit','buy') == OrderType.LIMIT_BUY
assert OrderType.from_type_side('limit','sell') == OrderType.LIMIT_SELL
assert OrderType.from_type_side('market','buy') == OrderType.MARKET_BUY
assert OrderType.from_type_side('market','sell') == OrderType.MARKET_SELL

In [ ]:
assert OrderType.LIMIT_BUY in OrderType.buy_types()
assert OrderType.MARKET_BUY in OrderType.buy_types()
assert OrderType.LIMIT_SELL in OrderType.sell_types()
assert OrderType.MARKET_SELL in OrderType.sell_types()

In [ ]:
assert OrderType.LIMIT_BUY.is_buy()
assert OrderType.MARKET_BUY.is_buy()
assert OrderType.LIMIT_SELL.is_sell()
assert OrderType.MARKET_SELL.is_sell()
OrderType.LIMIT_BUY.name, OrderType.LIMIT_BUY.value

### Order

In [ ]:
asset = Asset(c.LTC, c.USDT)
order = Order(
    exchange_id=c.PAPER, 
    asset=asset,
    price=250., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
Order.from_dict(order.to_dict())

### Order Manager

In [ ]:
asset = Asset(c.LTC, c.BTC)
exchange = load_exchange(c.PAPER)
exchange.balance = Balance(c.BTC, 5.0)
o1 = Order(exchange.id, asset, price=.1, quantity=1.0, order_type=OrderType.LIMIT_BUY)
o2 = Order(exchange.id, asset, price=.1, quantity=1.0, order_type=OrderType.LIMIT_SELL)
orders = {
    o1.id: o1,
    o2.id: o2
}
orders,exchange.balance

In [ ]:
resp = order_manager.place_order(exchange, orders[o1.id])
orders[o1.id] = resp
orders,exchange.balance

In [ ]:
resp = order_manager.place_order(exchange, orders[o2.id])
orders[o2.id] = resp
orders,exchange.balance

In [ ]:
limit_buy = order_manager.build_limit_buy_order(exchange, asset, price=.1, quantity=1.0)
limit_sell = order_manager.build_limit_sell_order(exchange, asset, price=.1, quantity=1.0)
market_buy = order_manager.build_market_buy_order(exchange, asset, quantity=1.0)
market_sell = order_manager.build_market_sell_order(exchange, asset, quantity=1.0)
orders = [limit_buy, limit_sell, market_buy, market_sell]
orders

In [ ]:
results = []
for order in orders:
    res = order_manager.place_order(exchange, order)
    results.append(res)
results

In [ ]:
exchange.balance

In [ ]:
exchange.orders

In [ ]:
#5e5b1bbbcc9e4edcbca2ec6c0663a1b4
exchange.fetch_orders(asset)

In [ ]:
exchange.fetch_order('57eb982b9afb49208b081a117022ffbe')

In [ ]:
updated_orders = exchange.fetch_orders(asset)
ex_order_ids = [order['id'] for order in updated_orders]
print(ex_order_ids)
updated_orders

In [ ]:
ex_orders = order_manager.get_orders(exchange, ex_order_ids)
print(ex_orders)

In [ ]:
type(ex_orders[0])

In [ ]:
print("PENDING", order_manager.get_pending_orders(ex_orders))
print("CANCELED", order_manager.get_canceled_orders(ex_orders))
print("FILLED", order_manager.get_filled_orders(ex_orders))

### Position

In [ ]:
asset = Asset(c.LTC, c.BTC)
pos = Position(asset, quantity=1, cost_price=250.0)
print("Asset", pos.asset.to_dict())
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

pos.update(-1, 200)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Go short
pos.update(-2, 150)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

# Close the short for a LOSS and go long (price went up, we had to pay $300 to cover our short)
# then we bought another share because we're newbs
pos.update(2, 300)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

In [ ]:
"""
Assume that an investor made the following consecutive fund purchases in a taxable account: 1,500 shares at $20, 1,000 shares at $10 and 1,250 shares at $8. The investor’s average cost basis is calculated by dividing $50,000/3,750 shares. The average cost is $13.33.

Suppose the investor then sells 1,000 shares of the fund at $19.

Gain/loss using average cost basis: ($19 - $13.33) x 1,000 shares = $5,667
"""

asset = Asset(c.LTC, c.BTC)
pos = Position(asset, quantity=1500, cost_price=20.0)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1000, txn_price=10)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
pos.update(txn_quantity=1250, txn_price=8)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)

last_cost = pos.cost_price
pos.update(txn_quantity=-1000, txn_price=19)
print("Value", pos.cost_value, "Quantity", pos.quantity, "Cost", pos.cost_price)
print("Profit", (19 - last_cost) * 1000)

In [ ]:
Position.from_dict(pos.to_dict()).to_dict()

### PerformanceTracker

In [ ]:
asset = Asset(c.BTC, c.USD)
perf = PerformanceTracker(starting_cash=5000, timeframe=Timeframe.ONE_MIN, store=None)
perf.to_dict()

In [ ]:
# Buy 1 BTC for $1000
pos1 = Position(asset, quantity=1, cost_price=1000.0)
positions = [pos1]
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of BTC increased $100
Position Return
    Return = .10
    PnL = $100
Cumulative Return
    Return = $100 / $5000 = .02
    PnL = $100
"""
pos1.latest_price = 1100
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of BTC increased again $400
Position Return
    Return = .5
    PnL = $500
Cumulative Return
    Return = $500 / $5000 = .1
    PnL = $500
"""
pos1.latest_price = 1500
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=4000.0,
    positions=positions
)
perf.to_dict()

In [ ]:
# Buy 1 ETH for $500
asset = Asset(c.ETH, c.USD)
pos2 = Position(asset, quantity=1, cost_price=500.0)
positions.append(pos2)
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

In [ ]:
"""
Value of ETH decreased $100
Position Return
    Return = -100/500
    PnL = -100
Cumulative Return
    Return = 400 / 5000 = .08
    PnL = 400
"""
pos2.latest_price = 400
perf.add_period(
    start=datetime.datetime.utcnow(),
    cash=3500.0,
    positions=positions
)
perf.to_dict()

In [ ]:
PerformanceTracker.from_dict(perf.to_dict())

### Portfolio

In [ ]:
starting_cash = 5000
exchange = load_exchange(c.PAPER)
perf = PerformanceTracker(starting_cash, Timeframe.ONE_MIN, store=None)
portfolio = Portfolio(starting_cash, perf)

In [ ]:
# Buy 1 BTC for $1000
asset = Asset(c.BTC, c.USD)
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1000., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
order

In [ ]:
portfolio.update(filled_orders=[order])
portfolio

In [ ]:
# Buy more BTC, price has risen $500
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=1, 
    order_type=OrderType.LIMIT_BUY
)
portfolio.update(filled_orders=[order])
portfolio

In [ ]:
# Sell all BTC at profit
order = Order(
    exchange_id=exchange.id, 
    asset=asset,
    price=1500., 
    quantity=2, 
    order_type=OrderType.LIMIT_SELL
)
portfolio.update(filled_orders=[order])
portfolio

In [ ]:
Portfolio.from_dict(portfolio.to_dict())

### Record

In [ ]:
feed = get_test_csv_feed(c.PAPER)

metrics = {
    'SMA': [1, 2, 3, 4],
    'RSI': [3, 4, 2, 1]
}

o1 = Order(c.PAPER, Asset(c.ETH, c.BTC), 5.0, 100.0, OrderType.LIMIT_BUY)
o2 = Order(c.PAPER, Asset(c.LTC, c.BTC), 5.0, 100.0, OrderType.LIMIT_BUY)
orders = {
    o1.id: o1,
    o2.id: o2
}

In [ ]:
root_dir = os.path.join(cfg.DATA_DIR, 'default')
store = FileStore(root_dir)

record = Record(config={}, portfolio=portfolio, balance=Balance(), store=store)
record.orders = orders
record.metrics = metrics
record.ohlcv = feed.history()
record.save()
record = Record.load(root_dir)

In [ ]:
print(record.config)
print(record.metrics)
print(record.balance)
print(record.orders)
print(record.ohlcv.head()[['time_utc','close']])
print(record.portfolio)

### Context

In [ ]:
feed = get_test_csv_feed(c.PAPER)
ctx = Context(DEFAULT_CONFIG, exchange, feed, record)

In [ ]:
ctx = Context.from_config(DEFAULT_CONFIG)
print(ctx.record.config)
print(ctx.record.metrics)
print(ctx.record.balance)
print(ctx.record.orders)
print(ctx.record.ohlcv)
print(ctx.record.portfolio)

### Strategy

In [ ]:
# https://www.backtrader.com/docu/quickstart/quickstart.html
# https://enigmampc.github.io/catalyst/beginner-tutorial.html#basics
from strategies.strategy import Strategy
from trading import order_manager

class MyStrategy(Strategy):
    def __init__(self):
        super().__init__()
    
    def log_all(self, orders, data, ctx, time_utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(time_utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)
    
    def handle_data(self, data, ctx):
        # Update metrics
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        
        # Add latest OHLCV row to record
        ctx.record.add_ohlcv(data)
        
        # Create orders
        orders = []
        asset = Asset(c.ETH, c.BTC)
        if random.random() > .5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, asset, price=0.1, quantity=1)
        else:
            order = order_manager.build_limit_sell_order(
                ctx.exchange, asset, price=0.1, quantity=1)
        orders.append(order)

        self.log_all(orders, data, ctx, data['time_utc'])
        
        return orders


In [ ]:
import shutil
from data.store import DATA_STORES, FILE_STORE
from data.feed import EXCHANGE_FEED, CSV_FEED

config = {
    'experiment': 'default',
    'exchange_id': c.PAPER,
    'cash_asset': c.BTC,
    'starting_cash': 1.0,
    'store': FILE_STORE,
    'feed': {
        'name': EXCHANGE_FEED,
        'fpath': os.path.join(cfg.DATA_DIR, 'default_feed.csv'),
        'symbols': ['ETH/BTC'],
        'timeframe': Timeframe.ONE_MIN.name,
        'start': '2018-01-10T08:00:00',
        'end': None,
    },
    'balance': {
        c.BTC: {'free': 1.0, 'used':0.0, 'total': 1.0},
        'free': {c.BTC: 1.0},
        'used': {c.BTC: 0.0},
        'total': {c.BTC: 1.0},
    }
}
shutil.rmtree(os.path.join(cfg.DATA_DIR, config['experiment']), ignore_errors=True)
ctx = Context.from_config(config)
s = MyStrategy()

In [ ]:
orders = s.process(ctx.feed.next(), ctx)

In [ ]:
resp = order_manager.place_orders(ctx.exchange, orders)
resp

In [ ]:
print("BEFORE")
print(ctx.record.balance)
# We're not updating the virtual balance, only the exchange 
# (which is fine until we want to have a multi-exchange algo)
print("AFTER Exchange Balance")
print(ctx.exchange.balance)

### Runner

In [105]:
import shutil
from data.store import DATA_STORES, FILE_STORE
from data.feed import EXCHANGE_FEED, CSV_FEED

config = {
    'experiment': 'default',
    'exchange_id': c.PAPER,
    'cash_asset': c.BTC,
    'starting_cash': 1.0,
    'store': FILE_STORE,
    'feed': {
        'name': EXCHANGE_FEED,
        'fpath': os.path.join(cfg.DATA_DIR, 'default_feed.csv'),
        'symbols': ['ETH/BTC'],
        'timeframe': Timeframe.ONE_MIN.name,
        'start': '2018-01-10T08:00:00',
        'end': None,
    },
    'balance': {
        c.BTC: {'free': 1.0, 'used':0.0, 'total': 1.0},
        'free': {c.BTC: 1.0},
        'used': {c.BTC: 0.0},
        'total': {c.BTC: 1.0},
    }
}

root = os.path.join(cfg.DATA_DIR, config['experiment'])
shutil.rmtree(root, ignore_errors=True)
mystrategy = MyStrategy()
context = Context.from_config(config)
punisher.punish(context, mystrategy)

Loading feed: .data/default_feed.csv
Downloading: ETH/BTC


---------------------------------------
Epoch 1 - Timestep: 2018-01-10T16:23:00
---------------------------------------
OHLCV
    O: 0.0924 | C: 0.0924 | V: 248.7 | T: 2018-01-10T16:23:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.1000 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 1.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00


Downloaded rows: 500
Punishing ...
{'experiment': 'default', 'exchange_id': 'paper', 'cash_asset': 'BTC', 'starting_cash': 1.0, 'store': 'CSV_STORE', 'feed': {'name': 'EXCHANGE_FEED', 'fpath': '.data/default_feed.csv', 'symbols': ['ETH/BTC'], 'timeframe': 'ONE_MIN', 'start': '2018-01-10T08:00:00', 'end': None}, 'balance': {'BTC': {'free': 1.0, 'used': 0.0, 'total': 1.0}, 'free': {'BTC': 1.0}, 'used': {'BTC': 0.0}, 'total': {'BTC': 1.0}}}


---------------------------------------
Epoch 2 - Timestep: 2018-01-10T16:24:00
---------------------------------------
OHLCV
    O: 0.0924 | C: 0.0924 | V: 332.6 | T: 2018-01-10T16:24:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.1000 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.9000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.9, 'used': 0.0, 'total': 0.9}
    ETH - {'free': 1.0, 'used': 0.0, 'total': 1.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 1, 'cost_price': 0.1, 'latest_price': 0.1}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 3 - Timestep: 2018-01-10T16:25:00
---------------------------------------
OHLCV
    O: 0.0924 | C: 0.0925 | V: 539.0 | T: 2018-01-10T16:25:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.1000 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.8000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.8, 'used': 0.0, 'total': 0.8}
    ETH - {'free': 2.0, 'used': 0.0, 't

    ETH - {'free': 4.0, 'used': 0.0, 'total': 4.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 4, 'cost_price': 0.1, 'latest_price': 0.1}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 16 - Timestep: 2018-01-10T16:38:00
---------------------------------------
OHLCV
    O: 0.0923 | C: 0.0922 | V: 470.1 | T: 2018-01-10T16:38:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.1000 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.5000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.5000000000000001, 'used': 0.0, 'total': 0.5000000000000001}
    ETH - {'free': 5.0, 'used': 0.0, 'total': 5.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 5, 'cost_price': 0.1, 'latest_price': 0.1}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 17 - Timestep: 2018-01-10T16:39:00
---------------------------------------
OHLCV
    O: 0.0922 | C: 0.0922 | V: 438.3 | T: 2018-01-10T16:39:00
ORDERS
    1: ETH/BTC | LIMIT_S

...

Balance is not sufficient to create order!


---------------------------------------
Epoch 66 - Timestep: 2018-01-10T17:28:00
---------------------------------------
OHLCV
    O: 0.0932 | C: 0.0933 | V: 250.4 | T: 2018-01-10T17:28:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.1000 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 1.0}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0, 'cost_price': 0.0, 'latest_price': 0.1}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 67 - Timestep: 2018-01-10T17:29:00
---------------------------------------
OHLCV
    O: 0.0933 | C: 0.0933 | V: 172.3 | T: 2018-01-10T17:29:00
ORDERS
    1: ETH/BTC | LIMIT_SELL | Price: 0.1000 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.9000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.9, 'used': 0.0, 'total': 0.9}
    ETH - {'free': 1.0, 'used': 0.0,

Balance is not sufficient to create order!


---------------------------------------
Epoch 69 - Timestep: 2018-01-10T17:31:00
---------------------------------------
OHLCV
    O: 0.0931 | C: 0.0934 | V: 402.8 | T: 2018-01-10T17:31:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.1000 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 1.0}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0, 'cost_price': 0.0, 'latest_price': 0.1}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 70 - Timestep: 2018-01-10T17:32:00
---------------------------------------
OHLCV
    O: 0.0934 | C: 0.0934 | V: 218.2 | T: 2018-01-10T17:32:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.1000 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.9000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 0.9, 'used': 0.0, 'total': 0.9}
    ETH - {'free': 1.0, 'used': 0.0, 

KeyboardInterrupt: 

### ChartDataProviders

In [102]:
import threading
from utils.dates import date_to_str

class RecordChartDataProvider():
    def __init__(self, root_dir, refresh_sec=5, t_minus=sys.maxsize):
        self.root_dir = root_dir
        self.refresh_sec = refresh_sec
        self.t_minus = t_minus
        self.thread = threading.Thread(target=self.update)
        self.thread.daemon = True
        self.record = Record.load(self.root_dir)

    def initialize(self):
        self.thread.start()

    def get_timeline(self):
        return self.get_ohlcv()['time_utc']

    def get_symbols(self):
        return self.record.portfolio.symbols

    def get_config(self):
        return self.record.config

    def get_ohlcv(self):
        """
        Returns dictionary:
            {'close': 0.077,
             'high': 0.0773,
             'low': 0.0771,
             'open': 0.0772,
             'time_utc': Timestamp('2018-01-08 22:22:00'),
             'volume': 222.514}
        """
        if abs(self.t_minus) >= len(self.record.ohlcv):
            return self.record.ohlcv
        return self.record.ohlcv.iloc[-self.t_minus:]

    def get_positions(self):
        positions = self.record.portfolio.positions
        return pd.DataFrame([p.to_dict() for p in positions])

    def get_positions_dct(self):
        positions = self.record.portfolio.positions
        dct = [p.to_dict() for p in positions]
        return dct

    def get_performance(self):
        return self.record.portfolio.perf

    def get_pnl(self):
        periods = self.record.portfolio.perf.periods
        return pd.DataFrame([
            [p['end_time'], p['pnl']] for p in periods
        ], columns=['time_utc','pnl'])

    def get_returns(self):
        periods = self.record.portfolio.perf.periods
        return pd.DataFrame([
            [p['end_time'], p['returns']] for p in periods
        ], columns=['time_utc','returns'])

    def get_balance(self):
        columns = ['coin', 'free', 'used', 'total']
        balance = self.record.balance
        dct = balance.to_dict()
        return pd.DataFrame(
            data=[
                [c, dct[c]['free'], dct[c]['used'], dct[c]['total']]
                for c in balance.currencies],
            columns=columns
        )

    def get_balance_dct(self):
        coins = self.record.balance.currencies
        dct = self.record.balance.to_dict()
        return [{
            'coin':c, 'free':dct[c]['free'],
            'used':dct[c]['used'], 'total':dct[c]['total']
        } for c in coins]

    def get_orders(self):
        columns = [
            'created', 'exchange', 'symbol', 'type',
            'price', 'quantity', 'filled', 'status'
        ]
        data = [
            [o.created_time, o.exchange_id, o.asset.symbol,
             o.order_type.name, o.price, o.quantity, o.filled_quantity,
             o.status.name] for o in self.record.orders.values()
        ]
        return pd.DataFrame(data=data, columns=columns)

    def get_orders_dct(self):
        return [{
            'created': date_to_str(o.created_time),
            'exchange': o.exchange_id,
            'symbol': o.asset.symbol,
            'type': o.order_type.name,
            'price': o.price,
            'quantity': o.quantity,
            'filled': o.filled_quantity,
            'status': o.status.name
            } for o in self.record.orders.values()
        ]

    def get_orders_hist(self):
        raise NotImplemented

    def get_metrics(self):
        return self.record.metrics

    def update(self):
        while True:
            print("Refreshing data")
            self.record = Record.load(self.root_dir)
            time.sleep(self.refresh_sec)


In [103]:
root = os.path.join(cfg.DATA_DIR, 'default')
rp = RecordChartDataProvider(root)

In [104]:
# TODO
print("OHLCV\n", rp.get_ohlcv())
print("\nPERFORMANCE\n", rp.get_performance())
print("\nRETURNS\n", rp.get_returns())
print("\nPNL\n", rp.get_pnl())

print("\nBALANCE\n", rp.get_balance())
print("\nBALANCE_DICT\n", rp.get_balance_dct())
print("\nPOSITIONS\n", rp.get_positions())
print("\nPOSITIONS_DICT\n", rp.get_positions_dct())

print("\nORDERS\n", rp.get_orders())
print("\nORDERS_DICT\n", rp.get_orders_dct())
print("\nMETRICS\n", rp.get_metrics())

OHLCV
                 open      high       low     close   volume  \
time_epoch                                                    
1515601380  0.092445  0.092470  0.092340  0.092447  248.701   
1515601440  0.092447  0.092450  0.092350  0.092443  332.587   
1515601500  0.092443  0.092639  0.092342  0.092452  539.003   
1515601560  0.092500  0.092580  0.092200  0.092206  321.022   
1515601620  0.092205  0.092730  0.092100  0.092199  656.431   
1515601680  0.092201  0.092500  0.092012  0.092117  343.741   
1515601740  0.092248  0.092285  0.092005  0.092223  397.709   
1515601800  0.092130  0.092683  0.092012  0.092199  400.649   
1515601860  0.092106  0.092373  0.092100  0.092128  270.245   
1515601920  0.092352  0.092683  0.092128  0.092495  315.674   
1515601980  0.092495  0.092496  0.092122  0.092415  483.976   
1515602040  0.092201  0.092421  0.092105  0.092276  379.561   

                      time_utc  
time_epoch                      
1515601380 2018-01-10 16:23:00  
1515601440 

### Charts

In [ ]:
feed = get_test_csv_feed(c.PAPER)
start = feed.history().iloc[0]['time_utc']
end = feed.history().iloc[-1]['time_utc']

In [ ]:
# Basic plot
utils.charts.plot_range(feed.history(), start, end, 'close')

### Dash

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

periods = record.portfolio.perf.periods
df = pd.DataFrame([
    [p['end_time'], p['pnl']] for p in periods
], columns=['time_utc','pnl'])
df

In [ ]:
positions = pd.DataFrame([p.to_dict() for p in record.portfolio.positions])
dct = [p.to_dict() for p in record.portfolio.positions]
{p['asset']: p for p in dct}

In [ ]:
cols = ['coin', 'free', 'used', 'total']
coins = b.currencies
dct = b.to_dict()
df = pd.DataFrame(
    [[c, dct[c]['free'], dct[c]['used'], dct[c]['total']] for c in coins],
    columns=cols
)
df